In [ ]:
!hostnamectl

In [ ]:
import kagglehub
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import random
from torch import optim

In [ ]:
torch.cuda.is_available()

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False


In [ ]:
class ModelReg(nn.Module):
    def __init__(self,input_dim, output_dim):
        super(ModelReg, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, self.input_dim + 10),
            nn.ReLU(),
            nn.Linear(self.input_dim + 10, self.input_dim + 10),
            nn.ReLU(),
            nn.Linear(self.input_dim + 10, self.input_dim),
            nn.ReLU(),
            nn.Linear(self.input_dim, self.input_dim // 2),
            nn.ReLU(),
            nn.Linear(self.input_dim // 2, self.output_dim)
        )
        
    def forward(self,x):
        out = self.layers(x)
        return out

In [ ]:
class ModelRegv2(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ModelRegv2, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, self.input_dim * 2 + 2),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(self.input_dim * 2 + 2, self.input_dim + 1),  # Исправлено
            nn.ReLU(),
            nn.BatchNorm1d(self.input_dim + 1),  # Исправлено
            nn.Linear(self.input_dim + 1, (self.input_dim + 1) // 2),
            nn.ReLU(),
            nn.Linear((self.input_dim + 1) // 2, self.output_dim)
        )
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
class ModelRegv4(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ModelRegv4, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, self.input_dim * 2),  # 31 -> 62
            nn.ReLU(),
            nn.BatchNorm1d(self.input_dim * 2),
            nn.Dropout(0.3),
            nn.Linear(self.input_dim * 2, self.input_dim),  # 62 -> 31
            nn.ReLU(),
            nn.BatchNorm1d(self.input_dim),
            nn.Dropout(0.3),
            nn.Linear(self.input_dim, self.input_dim // 2),  # 31 -> 15
            nn.ReLU(),
            nn.Linear(self.input_dim // 2, self.output_dim)  # 15 -> 1
        )
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
class ModelRegv3(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ModelRegv3, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, self.input_dim * 3),  
            nn.ReLU(),
            nn.Linear(self.input_dim * 3, self.input_dim * 2),  
            nn.ReLU(),
            nn.Linear(self.input_dim * 2, self.input_dim + 32),  
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout перед уменьшением размерности
            nn.Linear(self.input_dim + 32, self.input_dim // 2 + 16),
            nn.ReLU(),
            nn.Linear(self.input_dim // 2 + 16, self.input_dim // 4 + 8),
            nn.ReLU(),
            nn.Linear(self.input_dim // 4 + 8, self.output_dim)  # Выходной слой
        )

    def forward(self, x):
        return self.layers(x)


In [ ]:
 
#data
store_df = pd.read_csv('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/rossman/store.csv')
train_df = pd.read_csv('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/rossman/train.csv')

store_df['CompetitionDistance']= store_df['CompetitionDistance'].fillna(store_df['CompetitionDistance'].median())
store_df['CompetitionOpenSinceMonth'] = store_df['CompetitionOpenSinceMonth'].fillna(0)
store_df['CompetitionOpenSinceYear'] = store_df['CompetitionOpenSinceYear'].fillna(0)
store_df['HasCompetition'] = store_df['CompetitionOpenSinceMonth'] != 0
store_df['Promo2SinceWeek'] = store_df['Promo2SinceWeek'].fillna(0)
store_df['Promo2SinceYear'] = store_df['Promo2SinceYear'].fillna(0)
store_df['PromoInterval'] = store_df['PromoInterval'].fillna('None')

store_df = pd.get_dummies(store_df, columns=['StoreType', 'Assortment', 'PromoInterval'], prefix = ['StoreType', 'Assortment', 'PromoInterval'], dtype=int)
store_df['HasCompetition'] = store_df['HasCompetition'].astype(int)


fig,ax = plt.subplots(figsize = (20,20))
sns.heatmap(store_df.corr(), ax = ax, annot=True)
plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/store_df_corr.png')


fig,ax = plt.subplots(figsize = (20,20))
sns.heatmap(train_df.corr(numeric_only=True), ax = ax, annot=True)

plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/train_df_corr.png')

train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df = train_df.sort_values(by='Date')
train_df['StateHoliday'] = train_df['StateHoliday'].astype(str)
train_df = pd.get_dummies(train_df, columns=['StateHoliday'], prefix=['StateHoliday'], dtype=int)

train_df['Year'] = train_df['Date'].dt.year.astype('int64')
train_df['Month'] = train_df['Date'].dt.month.astype('int64')
train_df['Day'] = train_df['Date'].dt.day.astype('int64')

merged_df = pd.merge(train_df, store_df, on='Store', how='left')

#EDA

fig,ax = plt.subplots(figsize = (40,20))
sns.heatmap(merged_df.corr(), ax = ax, annot=True)
plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/merged_df_corr.png')

# Sales
plt.figure(figsize=(10, 5))
sns.histplot(merged_df['Sales'], kde=True)
plt.title(' Sales distribution')
plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/Sales_distribution.png')

merged_df.set_index('Date', inplace=True)
plt.figure(figsize=(14, 6))
merged_df['Sales'].resample('W').sum().plot()
plt.title('Sales by week')
plt.xlabel('Data')
plt.ylabel('Sales')
plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/Sales_by_week.png')

train_df = train_df.drop('Date', axis=1)

plt.figure(figsize=(8, 4))
sns.boxplot(x='Promo', y='Sales', data=merged_df)
plt.title('Impact of Promo on Sales')
plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/Promo_on_Sales.png')


plt.figure(figsize=(8, 4))
sns.boxplot(x='DayOfWeek', y='Sales', data=merged_df.reset_index())
plt.title('The Impact of Day of the Week on Sales')
plt.savefig('/home/ir739wb/ilyarekun/nn-lessons/nn-o-task-hypr-testing/results/Day_on_Sales.png')

plt.show()

length = merged_df.shape[0]
train_len = int(length * 0.7)
test_len = int(length * 0.15)
val_len = length - train_len - test_len

print(length - train_len - test_len - val_len)

train_df = merged_df.iloc[0:train_len]
test_df = merged_df.iloc[train_len:train_len + test_len]
val_df = merged_df.iloc[train_len + test_len:]
print(f'train_df {train_df.shape}\ntest_df {test_df.shape}\nval_df {val_df.shape}')

y_train = train_df['Sales']
X_train = train_df.drop('Sales', axis=1)
y_test = test_df['Sales']
X_test = test_df.drop('Sales', axis=1)
y_val = val_df['Sales']
X_val = val_df.drop('Sales', axis=1)


scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
X_val = torch.tensor(X_val,dtype=torch.float32)

y_train = torch.tensor(y_train.to_numpy(),dtype=torch.float32).reshape(-1,1)
y_test = torch.tensor(y_test.to_numpy(),dtype=torch.float32).reshape(-1,1)
y_val = torch.tensor(y_val.to_numpy(),dtype=torch.float32).reshape(-1,1)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
val_dataset = TensorDataset(X_val, y_val)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)



In [ ]:
print(np.isnan(X_train.numpy()).sum(), np.isinf(X_train.numpy()).sum())
print(np.isnan(y_train.numpy()).sum(), np.isinf(y_train.numpy()).sum())


In [ ]:
# Training loop 1

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
max_epochs = 100
learning_rate = 0.01
model = ModelRegv2(31, 1)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam лучше для устойчивости

# Metrics
train_loss_metr = []
val_loss_metr = []

for epoch in range(max_epochs):
    
    model.train()
    train_loss = 0.0
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)  # Суммируем loss для всех данных
    
    train_loss /= len(train_loader.dataset)  # Усредняем loss
    
    model.eval()
    val_loss = 0.0
        
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)  # Суммируем loss для всех данных
            
        val_loss /= len(val_loader.dataset)  # Усредняем loss
    
    # Запись метрик
    train_loss_metr.append(train_loss)
    val_loss_metr.append(val_loss)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    print('\n')


In [ ]:
epochs = range(1, len(train_loss_metr) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_metr, label='Train Loss', marker='o')
plt.plot(epochs, val_loss_metr, label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Train and Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig('./nn-o-task-hypr-testing/results/loss_over_epochsv4.png')
plt.show()


In [ ]:
train_df.Sales.describe()

In [ ]:
train_df.info()